# Preliminary Items

## Imports

In [ ]:
import os
import pandas as pd

import requests
import json

Enable colab pandas interactive displays

In [ ]:
from google.colab import data_table
%load_ext google.colab.data_table

## Mount Google Drive 

In [ ]:
currentWorkingDir = !pwd
defaultWorkingDir = "/content"

if ( currentWorkingDir[0] == defaultWorkingDir ):
  from google.colab import drive

  drive.mount('/content/drive')
      
  %cd "/content/drive/My Drive/Colab Notebooks/stock_portfolio/"
else:
  print("Currenting running app from: ")
  !pwd


Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/stock_portfolio


## Data Preparation

### Read in file

In [ ]:
symbol = "ccc"

In [ ]:
df_tweets = pd.read_csv("tweets_"+symbol+"_raw.csv")#usecols=list(range(21))[1:]) # usecols - don't pickup first column

In [ ]:
df_tweets

,id_str,created_at,place,source,text,coordinates,geo,is_quote_status,retweet_count,quote_count,reply_count,user_created_at,user_description,user_followers_count,user_following,user_friends_count,user_id_str,user_location,user_name,user_statuses_count
0,1335943138259570688,Mon Dec 07 13:43:59 +0000 2020,NaN,"<a href=""http://twitter.com/download/iphone"" r...",People who guh gym walk like dem build with Ca...,NaN,NaN,False,109,74,29,Wed May 30 19:04:30 +0000 2018,"Jamaican Journalist, Educator, MA Student, Clo...",21118,NaN,2304,1001902185876279297,Japan,Lij,38871
1,1335349902952554496,Sat Dec 05 22:26:41 +0000 2020,NaN,"<a href=""http://twitter.com/download/android"" ...",@emma_belle24 Well greatest thing you know the...,NaN,NaN,False,0,1,0,Thu Jul 30 12:20:00 +0000 2020,Carpenter by trade loves working with my hands...,1494,NaN,4532,1288811513856352256,NaN,D@n£¥`$ ©@®p£nt®¥🇯🇲,7271
2,1334257818560557056,Wed Dec 02 22:07:07 +0000 2020,"{'attributes': {}, 'bounding_box': {'coordinat...","<a href=""http://twitter.com/download/iphone"" r...",Lady just come in the taxi a Carib cement and ...,NaN,NaN,False,0,0,0,Sun Feb 26 18:22:54 +0000 2012,in captivity,301,NaN,770,504830904,Jamaica,Mugiwara Joey,11586
3,1333064466200715269,Sun Nov 29 15:05:10 +0000 2020,NaN,"<a href=""https://www.hootsuite.com"" rel=""nofol...","With COVID-19, many J’cans are struggling econ...",NaN,NaN,False,0,0,0,Fri Feb 14 02:30:47 +0000 2020,Jamaica's largest manufacturing company commit...,120,NaN,80,1228144478046838785,"Rockfort, Jamaica",CaribCementJa,261
4,1332141517369204737,Fri Nov 27 01:57:42 +0000 2020,NaN,"<a href=""http://twitter.com/download/iphone"" r...",CARIB CEMENT😭 bawl😭 https://t.co/iK8UMymMjY,NaN,NaN,True,0,0,0,Mon Dec 29 22:48:27 +0000 2014,TPWK (•Treat People With Kindess•) BLACKLIVESM...,1733,NaN,1189,2950199656,"Port Antonio, Jamaica",cinderella cause she broke homie,29994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,1336999661224292354,Thu Dec 10 11:42:14 +0000 2020,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Ye but we paying Carib Cement fi burn we tires...,NaN,NaN,True,0,0,1,Mon May 24 15:36:46 +0000 2010,Meeching,2188,NaN,1971,147607161,Still A Roam,Public Enemy #1,407390
726,1336998824729063424,Thu Dec 10 11:38:54 +0000 2020,NaN,"<a href=""http://twitter.com/download/android"" ...",Carib Cement pulled a Tom Sawyer on us lol htt...,NaN,NaN,True,0,0,0,Fri Mar 17 11:56:51 +0000 2017,NaN,1081,NaN,1141,842706293525811201,Jamaica,Chris Pinnock,44824
727,1336996834833657856,Thu Dec 10 11:31:00 +0000 2020,NaN,"<a href=""https://about.twitter.com/products/tw...",The Government is implementing a multimillion-...,NaN,NaN,False,6,16,1,Tue Apr 22 04:40:35 +0000 2008,Jamaica's Number 1 Daily Newspaper: Every Day ...,259082,NaN,312,14470793,"Kingston, Jamaica",Jamaica Observer,202690
728,1344691693543714824,Thu Dec 31 17:07:37 +0000 2020,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@King_Scxtt That’s why mi buy Carib Cement 😂,NaN,NaN,False,0,0,1,Mon May 02 02:55:08 +0000 2011,I do Entertainment #DonPedroRecords + @TannedJ...,8928,NaN,1438,291479786,Pon Di Gaza Til' Death,Juan Pedrito,373723


View number of rows and columns

In [ ]:
df_tweets.shape

(730, 20)

View column data types

In [ ]:
df_tweets.dtypes

id_str                    int64
created_at               object
place                    object
source                   object
text                     object
coordinates              object
geo                      object
is_quote_status            bool
retweet_count             int64
quote_count               int64
reply_count               int64
user_created_at          object
user_description         object
user_followers_count      int64
user_following          float64
user_friends_count        int64
user_id_str               int64
user_location            object
user_name                object
user_statuses_count       int64
dtype: object

### Select Data

Determine keywords that will determine relevant tweets

In [ ]:
df_tweets[df_tweets['user_name']!='CaribCementJa'].loc[:,['id_str', 'text', 'user_name']]

,id_str,text,user_name
0,1335943138259570688,People who guh gym walk like dem build with Ca...,Lij
1,1335349902952554496,@emma_belle24 Well greatest thing you know the...,D@n£¥`$ ©@®p£nt®¥🇯🇲
2,1334257818560557056,Lady just come in the taxi a Carib cement and ...,Mugiwara Joey
4,1332141517369204737,CARIB CEMENT😭 bawl😭 https://t.co/iK8UMymMjY,cinderella cause she broke homie
5,1332136781668438016,Boiled fry dumpling &gt; fried dumpling&gt; du...,Lij
...,...,...,...
724,1337000948904898560,"Ye but we could a get so much more, why not me...",Public Enemy #1
725,1336999661224292354,Ye but we paying Carib Cement fi burn we tires...,Public Enemy #1
726,1336998824729063424,Carib Cement pulled a Tom Sawyer on us lol htt...,Chris Pinnock
727,1336996834833657856,The Government is implementing a multimillion-...,Jamaica Observer


### Clean Data

Check columns for N/A values

In [ ]:
df_tweets.isna().sum()

id_str                    0
created_at                0
place                   701
source                    0
text                      0
coordinates             727
geo                     727
is_quote_status           0
retweet_count             0
quote_count               0
reply_count               0
user_created_at           0
user_description         80
user_followers_count      0
user_following          730
user_friends_count        0
user_id_str               0
user_location           130
user_name                 0
user_statuses_count       0
dtype: int64

Check columns for "0" as value

In [ ]:
(df_tweets == 0).sum()

id_str                    0
created_at                0
place                     0
source                    0
text                      0
coordinates               0
geo                       0
is_quote_status         663
retweet_count           492
quote_count             622
reply_count             542
user_created_at           0
user_description          0
user_followers_count      1
user_following            0
user_friends_count        3
user_id_str               0
user_location             0
user_name                 0
user_statuses_count       0
dtype: int64

#### Column: Text

In [ ]:
import string

In [ ]:
import re

Remove unnecessary objects like hashtags, mentions, links and punctuation.

##### Remove http links

In [ ]:
def remove_http (string):
    return re.sub(r'http\S+', '', string)

In [ ]:
df_tweets['text_clean'] = df_tweets['text'].apply(lambda x: remove_http(x))

In [ ]:
df_tweets[df_tweets['user_name']!='CaribCementJa'].loc[:,['id_str', 'text', 'text_clean']]

,id_str,text,text_clean
0,1335943138259570688,People who guh gym walk like dem build with Ca...,People who guh gym walk like dem build with Ca...
1,1335349902952554496,@emma_belle24 Well greatest thing you know the...,@emma_belle24 Well greatest thing you know the...
2,1334257818560557056,Lady just come in the taxi a Carib cement and ...,Lady just come in the taxi a Carib cement and ...
4,1332141517369204737,CARIB CEMENT😭 bawl😭 https://t.co/iK8UMymMjY,CARIB CEMENT😭 bawl😭
5,1332136781668438016,Boiled fry dumpling &gt; fried dumpling&gt; du...,Boiled fry dumpling &gt; fried dumpling&gt; du...
...,...,...,...
724,1337000948904898560,"Ye but we could a get so much more, why not me...","Ye but we could a get so much more, why not me..."
725,1336999661224292354,Ye but we paying Carib Cement fi burn we tires...,Ye but we paying Carib Cement fi burn we tires...
726,1336998824729063424,Carib Cement pulled a Tom Sawyer on us lol htt...,Carib Cement pulled a Tom Sawyer on us lol
727,1336996834833657856,The Government is implementing a multimillion-...,The Government is implementing a multimillion-...


##### Remove at mentions and hashtags

In [ ]:
def remove_mension_hashtags (string):
    return re.sub(r'(@|#)\w*', '', string)

In [ ]:
df_tweets['text_clean'] = df_tweets['text_clean'].apply(lambda x: remove_mension_hashtags(x))

In [ ]:
df_tweets[df_tweets['user_name']!='CaribCementJa'].loc[:,['id_str', 'text', 'text_clean']]

,id_str,text,text_clean
0,1335943138259570688,People who guh gym walk like dem build with Ca...,People who guh gym walk like dem build with Ca...
1,1335349902952554496,@emma_belle24 Well greatest thing you know the...,Well greatest thing you know the team he play...
2,1334257818560557056,Lady just come in the taxi a Carib cement and ...,Lady just come in the taxi a Carib cement and ...
4,1332141517369204737,CARIB CEMENT😭 bawl😭 https://t.co/iK8UMymMjY,CARIB CEMENT😭 bawl😭
5,1332136781668438016,Boiled fry dumpling &gt; fried dumpling&gt; du...,Boiled fry dumpling &gt; fried dumpling&gt; du...
...,...,...,...
724,1337000948904898560,"Ye but we could a get so much more, why not me...","Ye but we could a get so much more, why not me..."
725,1336999661224292354,Ye but we paying Carib Cement fi burn we tires...,Ye but we paying Carib Cement fi burn we tires...
726,1336998824729063424,Carib Cement pulled a Tom Sawyer on us lol htt...,Carib Cement pulled a Tom Sawyer on us lol
727,1336996834833657856,The Government is implementing a multimillion-...,The Government is implementing a multimillion-...


##### Remove punctuations

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

In [ ]:
df_tweets['text_clean'] = df_tweets['text_clean'].apply(lambda x: remove_punct(x))

In [ ]:
df_tweets[df_tweets['user_name']!='CaribCementJa'].loc[:,['id_str', 'text', 'text_clean']]

,id_str,text,text_clean
0,1335943138259570688,People who guh gym walk like dem build with Ca...,People who guh gym walk like dem build with Ca...
1,1335349902952554496,@emma_belle24 Well greatest thing you know the...,Well greatest thing you know the team he play...
2,1334257818560557056,Lady just come in the taxi a Carib cement and ...,Lady just come in the taxi a Carib cement and ...
4,1332141517369204737,CARIB CEMENT😭 bawl😭 https://t.co/iK8UMymMjY,CARIB CEMENT😭 bawl😭
5,1332136781668438016,Boiled fry dumpling &gt; fried dumpling&gt; du...,Boiled fry dumpling gt fried dumplinggt dumpli...
...,...,...,...
724,1337000948904898560,"Ye but we could a get so much more, why not me...",Ye but we could a get so much more why not mek...
725,1336999661224292354,Ye but we paying Carib Cement fi burn we tires...,Ye but we paying Carib Cement fi burn we tires...
726,1336998824729063424,Carib Cement pulled a Tom Sawyer on us lol htt...,Carib Cement pulled a Tom Sawyer on us lol
727,1336996834833657856,The Government is implementing a multimillion-...,The Government is implementing a multimilliond...


##### Replace emoji or emotions with text

In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 20.1MB/s 


In [ ]:
import emoji

In [ ]:
print(emoji.emojize('Python is :thumbs_up:'))

Python is 👍


In [ ]:
print(emoji.demojize('Python is 👍'))

Python is :thumbs_up:


In [ ]:
print(emoji.emojize("Python is fun :red_heart:"))

Python is fun ❤


In [ ]:
print(emoji.emojize("Python is fun :red_heart:",variant="emoji_type"))

Python is fun ❤️
